#COMPSCI 546: Applied Information Retrieval - Spring 2022 ([website](https://groups.cs.umass.edu/zamani/compsci-546-applied-information-retrieval-spring-2022/))
##Assignment 6: Clustering (Total : 100 points)

**Description**

This is a coding assignment where you will implement the k-means clustering algorithm to cluster passages. Basic proficiency in Python is recommended.  

**Instructions**

* To start working on the assignment, you would first need to save the notebook to your local Google Drive. For this purpose, you can click on *Copy to Drive* button. You can alternatively click the *Share* button located at the top right corner and click on *Copy Link* under *Get Link* to get a link and copy this notebook to your Google Drive.  

*   For questions with descriptive answers, please replace the text in the cell which states "Enter your answer here!" with your answer. If you are using mathematical notation in your answers, please define the variables.
*   You should implement all the functions yourself and should not use a library or tool for the computation.
*   For coding questions, you can add code where it says "enter code here" and execute the cell to print the output.
* To create the final pdf submission file, execute *Runtime->RunAll* from the menu to re-execute all the cells and then generate a PDF using *File->Print->Save as PDF*. Make sure that the generated PDF contains all the codes and printed outputs before submission.
To create the final python submission file, click on File->Download .py.


**Submission Details**

* Due data: Apr. 12, 2022 at 11:59 PM (EDT).
* The final PDF and python file must be uploaded on Moodle.
* After copying this notebook to your Google Drive, please paste a link to it below. Use the same process given above to generate a link. ***You will not recieve any credit if you don't paste the link!*** Make sure we can access the file. 
***LINK: *https://colab.research.google.com/drive/1BWlA9dS3IKgZRnhJkqT_DjkegfKf7idV?usp=sharing***

**Academic Honesty**

Please follow the guidelines under the *Collaboration and Help* section of the course website.     

# Download input files

Please execute the cell below to download the input files. 

In [ ]:

import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


import os
import zipfile
import numpy as np

download = drive.CreateFile({'id': '1NR5jMPJAJHD3flGEbckMr8WerfPtLtZ2'})
download.GetContentFile('HW06.zip')

with zipfile.ZipFile('HW06.zip', 'r') as zip_file:
    zip_file.extractall('./')
os.remove('HW06.zip')
# We will use hw05 as our working directory
os.chdir('HW06')

#Setting the input file
passage = "passages.tok.clean_kstem"
init_centroid_file = "centroid.txt"


# 1 : Initial Data Setup (25 points)

We use collection from the ANTIQUE  [https://arxiv.org/pdf/1905.08957.pdf] dataset for this assignment. As described in the previous assignments, this is a passage retrieval dataset. 

The description of the input files provided for this assignment is given below.

**Collection file**

Each row of the file consists of the following information:

*passage_id  passage_text*

The id and text information is tab separated. The passage text has been pre-processed to remove punctutation, tokenised and stemmed using the Krovetz stemmer. The terms in the passage text can be accessed by splitting the text based on space.

**Centroid vector values**

Each row is a tab separated entry where the first column is cluster id and the second column is the vector which is used to initialize the cluster centroid in the k-means Algorithm. 

In the cell below, collection information as vocababulary size, document frequency and passage count and initial centroid vector values have been given. This can be used in subsequent cells for computation. 



In [ ]:
''' 
In this function, load the initial centroid values for k clusters.  
Return Variables: 
init_centroid_vec - array where each element corresponds to the centroid vector. 
'''

# load initial centroid values
def load_centroid_init(init_centroid_file):
  init_centroid_vec = []
  for line in open(init_centroid_file):
   row = line.strip().split('\t')
   init_centroid_vec.append(np.fromstring(row[1][1:-1],sep=',').astype(float))
  return init_centroid_vec 


''' 
In this function, iterate through the input passage and load the data  
Return Variables: 
vocab - dict mapping word to an integer [0,len(vocab)]
df - dict mapping word to document frquency
num_passages - total number of passages in the input collection
docs-dict mapping each passage id to unique integer between [0,len(num_passages)]
'''
# load vocabulary
def load_vocab(passage):
  df = {}
  vocab = {}
  count = 0
  doc_count = 0
  num_passages = 0
  docs = {}
  for line in open(passage):  
   row = line.strip().split('\t')
   docs[row[0]] = doc_count
   doc_count+=1
   terms = row[1].split(' ')
   num_passages+=1
   for word in set(terms):
    if word not in df:
      df[word]=0
    df[word]+=1
   for word in terms:  
    if word not in vocab: 
      vocab[word]=count
      count+=1 

  return vocab,df,num_passages,docs     

init_centroid_vec = load_centroid_init(init_centroid_file)
vocab, df, num_passages, docs = load_vocab(passage)

print('Vocabulary Size : {0}'.format(len(vocab)))
print('Total Number of Passages: {0}'.format(num_passages))


Vocabulary Size : 3630
Total Number of Passages: 500


**Passage Representation**

In order to cluster passages, each passage has to be converted into a vector representation. For this assignment, we choose a $tf{\text -}idf$ representation. Every passage is represented by a vector with dimensionality equal to vocab_size. Each dimension in the vector corresponds to a word. The value of each dimension of the vector is the $tf{\text -}idf$ value of the word in that passage. These vectors can be stacked to create a matrix where each row of the matrix corresponds to a passage. 

Let the vocab_size be $n$ and num_passages be $m$.

In the cell below, implement the following:

1) Create an input data matrix $P = m \times n$  where $P_{i,j}=tf{\text -}idf(i,j)$. The definition of the $tf{\text -}idf(i,j)$ formulation is given below. 

$$ tf{\text -}idf(i,j) = ln(1+count(i,j)) ln(1 + \frac{|C|}{df(j)}) $$


$tf{\text -}idf(i,j)$: the  $tf{\text -}idf$ value of the word corresponding to integer $j$ and passage corresponding to integer $i$ as defined in vocab and docs data structures, respectively. 

$count(i,j)$ = number of times word corresponding to integer $j$ occurs in passage corresponding to integer $i$.

$df(j)$ - The document frequency of word corresponding to integer $j$ (i.e., the number of documents that contain the $j^{th}$ word.

$|C|$ - Total number of passages in the collection.


In [ ]:
from collections import Counter
''' 
In this function, create input matrix  
Return Variables: 
pass_matrix - matrix of shape m by n where each row corresponds to a passage and
each column to a word
'''
def create_input_matrix(passage,df,num_passages,vocab):
    #enter code here
    pass_matrix = []
    for line in open(passage):  
      row = line.strip().split('\t')
      words = row[1].split(' ')
      dict_count = Counter(words)
      vector = [0] * len(vocab)
      for k,v in dict_count.items():
        ipos = vocab[k]
        tfidf = np.log(1 + v) * np.log(1 + (num_passages/df[k]))
        vector[ipos] = tfidf  
      pass_matrix.append(vector)
    return pass_matrix

pass_matrix =  create_input_matrix(passage,df,num_passages,vocab)   

print('Shape of the input matrix : {0}'.format(np.shape(pass_matrix)))  


Shape of the input matrix : (500, 3630)


# 2 :  k-means Algorithm (35 points)

In this section you must implement the k-means clustering algorithm. The K-means function is called with $k=3$ ($k$ is the number of clusters) and the initial centroids have been set for you, the distance metric to be used is the squared Euclidean distance. Execute the algorithm for 30 iterations. 


In [ ]:
from collections import defaultdict
# centroid is a matrix instantiated with initial centroid values
''' 
In this function, centroid is instantiated with initial centroid values
Here we assume for k=3, the cluster ids are {0,1,2}   
Return Variables: 
centroid-matrix where rows correspond to clusters and columns correspond to words. 
'''
def  init_centers(vocab, k, init_centroid_vec):
  vocab_size = len(vocab)
  centroid = np.zeros((k, vocab_size), dtype='float')
  for i in range(len(init_centroid_vec)):
     centroid[i] = init_centroid_vec[i]
  return centroid   

''' 
In this function, implement kmeans algorithm   
Return Variables: 
final_cluster_assignment - array with cluster id values indexed by the passageid
 (integer mappings in docs)
num_passage_cluster_final - dict with cluster-id and key and number of elements 
in the cluster as value
'''
def kmeans(num_passages, pass_matrix, centroid, k):
   # enter code here
   n = 30 
   while (n>0):  
      c = 0
      final_dict = {}
      final_cluster_assignment = []
      for num in range(k):
        final_dict[num] = []
      for i in pass_matrix:
        tot_dist = []
        for j in centroid:
          a = np.array(i)
          b = np.array(j)
          dist = np.square(np.sum((a-b)**2)) #to find the euclidean distance
          tot_dist.append(dist)
        centr = np.where(tot_dist == np.amin(np.array(tot_dist)))[0]#min distance
        final_dict[int(centr)].append(c)
        final_cluster_assignment.append(int(centr))
        c += 1

      #recalculating the centroid
      for key, v in final_dict.items():
        a = []
        for pid in v:
          row = pass_matrix[pid]
          a.append(row)
        #new centroid values
        numpyA = np.array(a)
        centroid[key] = numpyA.mean(axis=0)
      n -= 1
      num_passage_cluster_final = defaultdict()

      for k1, v1 in final_dict.items():
        num_passage_cluster_final[k1] = len(v1)

   return final_cluster_assignment,num_passage_cluster_final 

centroid = init_centers(vocab, 3, init_centroid_vec)
final_cluster_assignment, num_passage_cluster_final = kmeans(num_passages, 
                                                    pass_matrix, centroid, 3)

print('Number of initial centroids : {0}'.format(len(centroid)) )

# print number of elements in each cluster
''' 
 Hint: The values lies within the interval [360,370] for the largest cluster, 
 [120,130] for the second largest cluster and [0,10] for the smallest cluster. 
'''
for cid,cnum in num_passage_cluster_final.items():
   print(cid, cnum)  



Number of initial centroids : 3
0 363
1 128
2 9


# 3 :  Evaluation (30 points)

In this section, you must implement two evaluation metrics: 

1) IntraCluster Similarity Metric - Average Diameter 
Distance of each cluster $S$

$$Sim(S) = \frac{1}{|S| (|S|-1)} \sum_{x,y \in S,x \neq y} dist(x,y)$$

$|S|$ - number of passages assigned to cluster $S$

$dist(x,y)$ - the squared euclidean distance between passages $x$ and $y$.


2) Intercluster Similarity Metric - 
Average Linkage Distance between a pair of clusters $S,T$

$$Sim(S,T) = \frac{1}{|S| |T|} \sum_{x \in S,y \in T} dist(x,y)$$

$|S|$ - number of passages assigned to cluster $S$

$|T|$ - number of passages assigned to cluster $T$

$dist(x,y)$ - the squared euclidean distance between passages $x$ and $y$.


You have to calculate and print out the IntraCluster Similarity Metric for every cluster and Intercluster Similarity Metric for every cluster pair. This is illustrated below with examples. 

For IntraCluster metric, Average Diameter Distance for cluster (a, b, c),
the value can be calculated as follows :$\frac{dist(a, b)+dist(a, c)+dist(b, a)+dist(b, c)+ dist(c, a)+ dist(c, b)}{3*2} $ 

For InterCluster metric, Average Linkage Distance for clusters (a, b, c) and (d, e) , the value can be calculated as follows:  $\frac{dist(a, d)+dist(a, e)+ dist(b, d)+ dist(b, e)+ dist (c, d)+ dist(c, e)}{3*2}$



In [ ]:
from typing import KeysView
''' 
In this function, implement intracluster similarity metric Average Diameter 
Distance   
Return Variables: 
sim_score - dict with metric score corresponding to each of the three clusters
'''

def average_diameter_dist(num_passage_cluster_final,final_cluster_assignment,
                          pass_matrix):
    # enter code here
    sim_score = defaultdict()
    for key,v in num_passage_cluster_final.items():
      dist_sum = 0
      opmatrix = []
      for i in range(0,len(final_cluster_assignment)):
        if final_cluster_assignment[i] == key:
          opmatrix.append(pass_matrix[i])

      for i in range(0, len(opmatrix)):
        for j in range(0, len(opmatrix)):
          if i != j:
            a = np.array([opmatrix[i]])
            b = np.array([opmatrix[j]])
            dist_sum += np.sum(np.square((a-b)))
      sim = dist_sum/(v*(v-1))
      sim_score[key] = sim
    return sim_score

''' 
In this function, implement intercluster similarity metric Average Linkage Distance   
Return Variables: 
sim_score - dict with metric score corresponding to each pair of clusters
'''
def average_linkage_dist(num_passage_cluster_final,final_cluster_assignment,
                         pass_matrix):
    # enter code here
    #num of cluster pairs
    sim_score = defaultdict(float)
    pairs = []
    cluster_pass_matrix = defaultdict(list)
    for i in range(0, len(num_passage_cluster_final)):
      for j in range(i+1, len(num_passage_cluster_final)):
        pairs.append((i,j))
      for x in range(0, len(final_cluster_assignment)): #setting up pass_matrix for each cluster ID
        if final_cluster_assignment[x] == i:
          cluster_pass_matrix[i].append(pass_matrix[x])

    for a in pairs:
      first = a[0]
      secon = a[1]
      dist_sum = 0
      for fir in range(0, num_passage_cluster_final[first]):
        a = np.array([cluster_pass_matrix[first][fir]])
        for sec in range(0, num_passage_cluster_final[secon]):        
          b = np.array([cluster_pass_matrix[secon][sec]])
          dist_sum += np.sum(np.square((a-b)))
      p1 = num_passage_cluster_final[first]
      p2 = num_passage_cluster_final[secon]
      sim = dist_sum/(p1*p2)
      sim_score[(first, secon)] = sim

    return sim_score

intra_score  = average_diameter_dist(num_passage_cluster_final,
                                     final_cluster_assignment,pass_matrix) 
inter_score =  average_linkage_dist(num_passage_cluster_final,
                                    final_cluster_assignment,pass_matrix)  

# print intracluster scores for very cluster
'''
Hint: The value lies within the intervals [310,320] for the largest cluster,
[1135,1145] for the second largest cluster and [3860,3870] for the smallest cluster.
'''
for cid,score in intra_score.items():
   print(cid, score) 

# print intercluster scores for every pair of cluster
'''
Hint: The value lies within the intervals [740,750] between largest and second 
largest clusters.
[2260,2270] between largest and smallest clusters and 
[2590,2600] between second largest and smallest cluster
'''
for cid_pair,score in inter_score.items():
   print(cid_pair, score)    

0 319.1771238766518
1 1140.0375066528834
2 3867.7247873603037
(0, 1) 744.3454376966359
(0, 2) 2268.718168712698
(1, 2) 2598.1074296167694


# 4 : k-Means Clustering (10 points)

**Question 4.1:** For this assignment, we used a predefined number of clusters, $k$. Describe a method which can be used to get a good approximation for this value for a dataset.   (5 points)



ANSWER: One method which can be used to get a good approximation of k is to use the 'Elbow Curve Method'. In this method the following steps are performed -

1. Choose a range of k values, say from 1-10.
2. For each of the k values, perform K-means clustering and calculate the average of the squared distances from the cluster centroids of the respective clusters for all the data points.
3. For each value of k, plot the distance calculated. 
4. The plot obtained will have a curve that looks like an elbow, from where, the distance value suddenly falls flat for increasing values of k.
5. The elbow point of k is chosen to be the optimal k for the given dataset.

**Question 4.2:** In this assignment, the initial centroids are given. The K-Means clustering output depends on the initial seeds. Therefore, different random seed selection would lead to different clustering outcomes. Describe a seed selection method that can lead to better K-Means clustering. (5 points)

ANSWER: There are many ways to initialize the centroids of the clusters in k-means clustering. One of the method that can lead to better k-means clustering is 'k-means++' or 'KMPP'. In this method, the first centre is chosen randomly from the given dataset. Then, the second centre is also chosen randomly from the dataset, but, the probability of selection of the datapoint is proportional to the euclidian distance of it to that of the 1st centre. Then, the third centre is again chosen randomly with the probability of selection proportional to the euclidian distance of the data point to the nearest of the previous two centres. Similarly, in the same way, subsequent centres can be calculated. This way, the k centres for the clusters can be initialized. 